In [1]:
# import required packages like flask, sqlite3, flask sessions used for login
from flask import Flask, render_template, request, session
from flask_session import Session
import pyodbc
from functools import wraps
from flask import session, redirect, url_for
import hashlib

# login checker function
def login_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if 'logged_in' not in session or not session['logged_in']:
            return redirect(url_for('index', next=request.url))
        return f(*args, **kwargs)
    return decorated_function

# get connected to the msaccess file
db_driver = '{Microsoft Access Driver (*.mdb, *.accdb)}'
db_path = './inventory.accdb'
conn_str = (rf'DRIVER={db_driver};'rf'DBQ={db_path};')

# setting up flask session
global msg
app = Flask(__name__)
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

# flask route used to go to index or home page


@app.route("/")
def index():
    title = "Login Page"
    data = {
        'title': title
    }
    # the title for the html template page is passed via data variable - this data varibale is called flask varibale
    return render_template("index.html", data=data)

# flask route for login page

@app.route("/login", methods=["POST"])
def login():
    title = "Login Page"
    # if method is post then only executes remaining steps
    if request.method == 'POST':
        try:
            # fetch usermail and pwd from login form
            # encrypt pwd with sha256 and cross verify the usermail and encrypted pwd with values in database
            usermail = request.form['usermail']
            userpwd = hashlib.sha256(
                request.form['password'].encode()).hexdigest()
            con = pyodbc.connect(conn_str)
            cur = con.cursor()
            mysql = "select * from users where emailid='"+usermail+"' and password='"+userpwd+"'"
            # cur.execute('select * from users')
            cur.execute(mysql)
            rows = cur.fetchall()
            if(rows):
                msg = 'Logged in Successfully.'
                head_to = 'dashboard'
                session["userid"] = rows[0][0]
                session["logged_in"] = True
                session["username"] = rows[0][1]
            else:
                msg = 'Invalid Credentials'
                head_to = 'login'
                msg_code = 400
                # if credentials match assign a random icon as profile photo and set login session
        except:
            con.rollback()
            head_to = 'login'
            msg = "Error in connecting to database"
            msg_code = 404

        finally:
            # pass the messgae to the html template
            data = {
                "title": title,
                "msg": msg,
                "redirect_url": head_to
            }
            return render_template("login.html", data=data)
            con.close()

# flask route to add product, only post method
@app.route('/add_product', methods=['POST'])
def add_product():
    # fetch form data
    product_name = request.form['productName']
    description = request.form['productDescription']
    price = request.form['productPrice']
    quantity = request.form['productQuantity']
    supplied_by = request.form['productSupplier']

    try:
        # Connect to the database
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        # SQL Query to insert data
        insert_query = '''
            INSERT INTO products (productname, description, price, quantity, supplierid)
            VALUES (?, ?, ?, ?, ?)
        '''
        cursor.execute(insert_query, (product_name, description, float(price), int(quantity), supplied_by))
        conn.commit()

    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle the error as needed
    finally:
        # Ensure that connection is closed
        if conn:
            conn.close()

    return redirect(url_for('products'))

# flaks route for logout page
@app.route("/logout")
def logout():
    title = "Logout Page"
    data = {
        'title': title,
        'msg': 'You have been logged out'
    }
    # logout msg is sent to html page and session are cleared
    session.pop('logged_in', None)
    return render_template('logout.html', data=data)

#flask route to call dashboard page
@app.route("/dashboard")
@login_required
def dashboard():
    title = "Dashboard Page"
    data = {
        'title' : title,
        'active_link' : 'dashboard_link'
    }
    return render_template("dashboard.html", data=data)

#flask route to call products page
@app.route("/products")
@login_required
def products():
    try:
        # get database connection to fetch products
        con = pyodbc.connect(conn_str)
        cur = con.cursor()
        rows = []
        mysql = "select * from products inner join suppliers on products.supplierid=suppliers.supplierid"
        cur.execute(mysql)
        rows = cur.fetchall()
        print(rows)
        if(rows):
            msg_code = 200
        else:
            msg_code = 400
        mysql = "select * from suppliers"
        cur.execute(mysql)
        suppliers_rows = cur.fetchall()
    except Exception as e:
        print(e)
        con.rollback()
        msg_code = 404

    finally:
        # pass the messgae to the html template
        title = "Products Page"
        data = {
            'title' : title,
            'active_link' : 'manage_products',
            'products_data' : rows,
            'msg_code': msg_code,
            'suppliers_rows': suppliers_rows
        }
        con.close()
        return render_template("products.html", data=data)

#flask route to call edit product page
@app.route("/edit_product/<action>/<product_id>", methods=['GET'])
@login_required
def edit_product(action, product_id):
    title = "Managing Product"
    product_data = []
    redirect_url = "edit_product.html"
    try:
        print(action,product_id)
        # get connected to database
        con = pyodbc.connect(conn_str)
        cur = con.cursor()
        userid = session["userid"]
        # if action is delete, delete the product from database
        if(action == "delete"):
            mysql = "delete from products where productid="+str(product_id)
            print(mysql)
            cur.execute(mysql)
            msg = "Product deleted successfully"
            con.commit()
            # if action is edit, redirect to another html page and fill forms info of the product
        elif(action == "edit"):
            cur.execute(
                "select * from products where productid="+str(product_id))
            product_data = cur.fetchone()
            msg = "Updating Product"
            redirect_url = "updating_product.html"
        
    except Exception as e:
        print(e)
        con.rollback()
        msg = "Unable to perform operation"
        # send the fetched info to the front end page
    finally:
        data = {
            "title": title,
            "msg": msg,
            "products_data": product_data
        }
        print(data)
        return render_template(redirect_url, data=data)
        con.close()

#flask route to update product table
@app.route("/update_product_table", methods=['GET','POST'])
@login_required
def update_product_table():
    try:
        # get connected to database
        con = pyodbc.connect(conn_str)
        cur = con.cursor()
        productid = request.form["productid"]
        productname = request.form["productname"]
        productdesc = request.form["productdesc"]
        productprice = request.form["productprice"]
        productquantity = request.form["productquantity"]
        print(productid)
        mysql = "update products set productname='"+productname+"', description='"+productdesc+"',price='"+productprice+"' , quantity='"+productquantity+"' where productid="+str(productid)
        print(mysql)
        cur.execute(mysql)
        msg = productname+" updated successfully"
        con.commit()

    except Exception as e:
        print(e)
        con.rollback()
        msg = "Unable to perform operation"
        # send the fetched info to the front end page
    finally:
        data = {
            "title": 'Saving data',
            "msg": msg
        }
        return render_template('edit_product.html', data=data)
        con.close()


#flask route to call suppliers page
@app.route("/suppliers")
@login_required
def suppliers():
    try:
        # get database connection to fetch suppliers
        con = pyodbc.connect(conn_str)
        cur = con.cursor()
        rows = []
        mysql = "select * from suppliers"
        cur.execute(mysql)
        rows = cur.fetchall()
        print(rows)
        if(rows):
            msg_code = 200
        else:
            msg_code = 400
    except Exception as e:
        print(e)
        con.rollback()
        msg_code = 404

    finally:
        # pass the messgae to the html template
        title = "Suppliers Page"
        data = {
            'title' : title,
            'active_link' : 'manage_suppliers',
            'suppliers_data' : rows,
            'msg_code': msg_code
        }
        con.close()
        return render_template("suppliers.html", data=data)

#flask route to call edit supplier page
@app.route("/edit_supplier/<action>/<supplier_id>", methods=['GET'])
@login_required
def edit_supplier(action, supplier_id):
    title = "Managing Supplier"
    supplier_data = []
    redirect_url = "edit_supplier.html"
    try:
        print(action,supplier_id)
        # get connected to database
        con = pyodbc.connect(conn_str)
        cur = con.cursor()
        userid = session["userid"]
        # if action is delete, delete supplier from database
        if(action == "delete"):
            mysql = "delete from suppliers where supplierid="+str(supplier_id)
            print(mysql)
            cur.execute(mysql)
            msg = "Supplier deleted successfully"
            # if action is edit, redirect to another html page and fill forms info of the supplier
        elif(action == "edit"):
            cur.execute(
                "select * from suppliers where supplierid="+str(supplier_id))
            supplier_data = cur.fetchone()
            msg = "Editing Supplier"
            redirect_url = "updating_supplier.html"
        con.commit()
    except Exception as e:
        print(e)
        con.rollback()
        msg = "Unable to perform operation"
        # send the fetched info to the front end page
    finally:
        data = {
            "title": title,
            "msg": msg,
            "supplier_data": supplier_data
        }
        print(data)
        return render_template(redirect_url, data=data)
        con.close()

#flask route to update supplier table
@app.route("/update_supplier_table", methods=['GET','POST'])
@login_required
def update_supplier_table():
    try:
        # get connected to database
        con = pyodbc.connect(conn_str)
        cur = con.cursor()
        supplierid = request.form["supplierid"]
        suppliername = request.form["suppliername"]
        contactinfo = request.form["contactinfo"]
        mysql = "update suppliers set suppliername='"+suppliername+"', contactinfo='"+contactinfo+"' where supplierid="+str(supplierid)
        print(mysql)
        cur.execute(mysql)
        msg = suppliername+" updated successfully"
        con.commit()

    except Exception as e:
        print(e)
        con.rollback()
        msg = "Unable to perform operation"
        # send the fetched info to the front end page
    finally:
        data = {
            "title": 'Saving data',
            "msg": msg
        }
        return render_template('edit_supplier.html', data=data)
        con.close()


# main flask call, this calls the routing method and starts the server
# debug is set to true to start the debug log
if __name__ == "__main__":
    app.run(debug=True)


ModuleNotFoundError: No module named 'flask'

In [ ]:
# Reduce the number of estimators in Random Forest to prevent memory issues
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)

# Retrain models after memory optimization

# Random Forest
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
rf_report = classification_report(y_test, rf_preds, output_dict=True)

# XGBoost (optimized for memory)
xgb_model = XGBClassifier(n_estimators=50, max_depth=10, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_preds)
xgb_report = classification_report(y_test, xgb_preds, output_dict=True)

# Neural Network (MLP) with fewer neurons
mlp_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=200, random_state=42)
mlp_model.fit(X_train, y_train)
mlp_preds = mlp_model.predict(X_test)
mlp_accuracy = accuracy_score(y_test, mlp_preds)
mlp_report = classification_report(y_test, mlp_preds, output_dict=True)

# Display updated model evaluation results
rf_accuracy, xgb_accuracy, mlp_accuracy, rf_report, xgb_report, mlp_report


In [ ]:
# Drop redundant column in Cancer Data
cancer_df.drop(columns=['Unnamed: 32', 'id'], inplace=True, errors='ignore')

# Encode categorical variables in all datasets
from sklearn.preprocessing import LabelEncoder

# Encode Cancer Data (diagnosis: M/B to 1/0)
cancer_df['diagnosis'] = cancer_df['diagnosis'].map({'M': 1, 'B': 0})

# Encode Obesity Data (categorical columns)
obesity_categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad']
for col in obesity_categorical_cols:
    le = LabelEncoder()
    obesity_df[col] = le.fit_transform(obesity_df[col])

# Encode Healthcare Data (categorical variables)
healthcare_categorical_cols = ['Gender', 'Blood Type', 'Medical Condition', 'Admission Type', 'Test Results', 'Medication']
for col in healthcare_categorical_cols:
    le = LabelEncoder()
    healthcare_df[col] = le.fit_transform(healthcare_df[col])

# Display processed datasets
import ace_tools as tools
tools.display_dataframe_to_user(name="Processed Cancer Data", dataframe=cancer_df)
tools.display_dataframe_to_user(name="Processed Diabetes Data", dataframe=diabetes_df)
tools.display_dataframe_to_user(name="Processed Obesity Data", dataframe=obesity_df)
tools.display_dataframe_to_user(name="Processed Healthcare Data", dataframe=healthcare_df)
